In [1]:
import dash
import dash_daq as daq
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

import flask
import pandas as pd
import time
import os

import numpy as np  
import pandas as pd  
from pandas_datareader import data as wb 
import matplotlib.pyplot as plt  
from scipy.stats import norm
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [2]:
server = flask.Flask('app')
server.secret_key = os.environ.get('secret_key', 'secret')

df = pd.read_csv('coins.csv')

app = dash.Dash('app', server=server)
app.css.config.serve_locally = True
app.scripts.config.serve_locally = False
dcc._js_dist[0]['external_url'] = 'https://cdn.plot.ly/plotly-basic-latest.min.js'
app.layout = html.Div([
    html.H1('Optimal Buy Time For Top 5 Cryptocurrencies',id='my-heading'),
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'BTC-USD', 'value': 'BTC'},
            {'label': 'ETH-USD', 'value': 'ETH'},
            {'label': 'XRP-USD', 'value': 'XRP'},
            {'label': 'LTC-USD', 'value': 'LTC'},
            {'label': 'USDT-USD', 'value': 'USDT'}
        ],
        value='BTC'
    ),
    dcc.Graph(id='my-graph'),
    dcc.Graph(id='new-graph')
], className="container")

In [3]:
colors = dict(
    background='#000000',
    text='#D3D3D3'
)

@app.callback(Output('my-graph', 'figure'),
              [Input('my-dropdown', 'value')])

def update_graph(selected_dropdown_value):
    dff = df.loc[:,selected_dropdown_value]
    a = dff.values.tolist()
    minimum = a.index(min(a))
    layout = go.Layout(autosize=True,
               margin={'t': 50, 'l': 50, 'r': 5},
                       font=dict(family='Courier New, monospace', size=15, color='#ffffff'),
               plot_bgcolor=colors['background'],
               paper_bgcolor=colors['background'],
#                font={'color': colors['text']},
    showlegend=False,title = 'Price for coming 12 days',
    annotations=[
        dict(
            x=minimum,
            y=a[minimum],
            xref='x',
            yref='y',
            text='Best Day',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        )],xaxis = dict(
      title = "Days",
    ),
    yaxis = dict(
      title = "Closing Price",
    ) )     
    data= [{
            'y': dff.values,
            'line': {
                'width': 3,
                'shape': 'spline'
            }
        }]
    return {'data':data,'layout':layout}  

In [ ]:
@app.callback(Output('new-graph', 'figure'),
        [Input('my-dropdown', 'value')])

def update_graph(selected_dropdown_value):
    ticker = selected_dropdown_value+"-USD"
    data = pd.DataFrame()
    data = wb.DataReader(ticker, data_source='yahoo', start='2012-01-01',end='2018-03-31')['Adj Close']
    log_returns = np.log(1 + data.pct_change())
    log_returns.tail()
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5 * var)
    stdev = log_returns.std()
    Z = norm.ppf(np.random.rand(10,2))
    t_intervals = 30
    iterations = 500
    daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
    S0 = data.iloc[-1]
    price_list = np.zeros_like(daily_returns)
    price_list[0]=S0
    for t in range(1, t_intervals):
        price_list[t] = price_list[t - 1] * daily_returns[t]
    price_list = price_list.transpose()

    layout = dict(title = 'Monte Carlo Simulation',
                  font=dict(family='Courier New, monospace', size=15, color='#ffffff'),
              xaxis = dict(title = 'Days'),
              yaxis = dict(title = 'Closing Price'),
              showlegend=False,autosize=True,
               margin={'t': 50, 'l': 50, 'r': 5},
               plot_bgcolor=colors['background'],
               paper_bgcolor=colors['background'],
#                font={'color': colors['text']}
             )
    data = []
    for i in range(500):
        data.append(go.Scatter(y=price_list[i]))

    return {
        'data': data,
        'layout': layout
    }

In [ ]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2019 02:34:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:34:15] "GET /assets/stylesheet.css?m=1554668285.12001 HTTP/1.1" 304 -
127.0.0.1 - - [08/Apr/2019 02:34:15] "GET /assets/plotly_ga.js?m=1541374912.0 HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:34:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:34:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:34:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:34:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:35:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:36:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:36:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:36:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 02:40:28] "

127.0.0.1 - - [08/Apr/2019 05:13:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 05:13:22] "POST /_dash-update-component HTTP/1.1" 200 -
